## 1. 環境確認

In [ ]:
import sys
import os

# CPU情報
cpu_count = os.cpu_count() or 8
print(f"利用可能CPU: {cpu_count} cores")
print(f"並列処理で高速化します")

## 2. モジュールインポート

In [ ]:
from pathlib import Path
import pandas as pd

# プロジェクトルートをパスに追加
sys.path.append('/workspace/atma_22_ca/')

from configs.config import *
from src.preprocess import preprocess_train_crops, verify_crops

print(f"✅ モジュールインポート完了")
print(f"   入力: {DIR_IMAGE}")
print(f"   メタ: {FILE_TRAIN_META}")

## 3. パス設定と事前チェック

In [ ]:
# パス設定
csv_path = Path(FILE_TRAIN_META)
image_dir = Path(DIR_IMAGE)
output_dir = Path(DIR_INTERIM) / 'train_crops'

print(f"📋 パス確認:")
print(f"   メタCSV: {csv_path}")
print(f"   画像Dir: {image_dir}")
print(f"   出力Dir: {output_dir}")

# データ数確認
df_train = pd.read_csv(csv_path)
print(f"\n📊 学習データ: {len(df_train):,}サンプル")
print(f"   ユニーク選手数: {df_train['label_id'].nunique()}")
print(f"   Quarter数: {df_train['quarter'].nunique()}")

# 既存チェック
if output_dir.exists():
    existing_count = len(list(output_dir.glob('*.jpg')))
    if existing_count > 0:
        print(f"\n⚠️ 既にクロップ済みデータが存在します:")
        print(f"   {output_dir}")
        print(f"   ファイル数: {existing_count:,}")
        print(f"\n   ※ 再実行する場合は、既存ディレクトリを削除してください")
    else:
        print(f"\n✅ 出力ディレクトリは空です")
else:
    print(f"\n✅ 新規作成します: {output_dir}")

## 4. クロップ実行

### 処理詳細
- **padding_ratio**: 0.1（BBoxサイズの10%パディング）
- **並列処理**: CPU全コアを使用
- **JPEG品質**: 95%（高品質）
- **所要時間**: 約5-10分（データ量・CPU性能による）

In [ ]:
%%time
# クロップ処理実行
print("="*80)
print("🎯 学習データ前処理（BBox切り出し）開始")
print("="*80)

preprocess_train_crops(
    csv_path=csv_path,
    image_dir=image_dir,
    output_dir=output_dir,
    padding_ratio=0.1,  # RedBullと同じ10%パディング
    num_workers=None,   # CPU数に応じて自動設定
)

print("\n" + "="*80)
print("✅ クロップ処理完了")
print("="*80)

## 5. 結果検証

In [ ]:
import cv2

In [ ]:
def verify_crops(csv_path: Path, crop_dir: Path, sample_size: int = 10):
    """クロップ結果の検証
    
    Args:
        csv_path: メタデータCSVパス
        crop_dir: クロップ画像ディレクトリ
        sample_size: 検証するサンプル数
    """
    df = pd.read_csv(str(csv_path))
    print(f"\n🔍 クロップ検証 ({sample_size}サンプル):")
    
    missing_count = 0
    for idx in df.index[:sample_size]:
        crop_path = crop_dir / f"{idx}.jpg"
        if not crop_path.exists():
            print(f"   ⚠️ 欠損: {idx}")
            missing_count += 1
        else:
            img = cv2.imread(str(crop_path))
            if img is None:
                print(f"   ⚠️ 読み込み失敗: {idx}")
                missing_count += 1
            else:
                h, w = img.shape[:2]
                print(f"   ✓ {idx}: {w}x{h}px")
    
    if missing_count == 0:
        print("   ✅ 全サンプル正常")
    else:
        print(f"   ⚠️ {missing_count}件の問題を検出")

In [ ]:
# クロップ結果を検証
verify_crops(csv_path, output_dir, sample_size=20)

# 全体統計
total_files = len(list(output_dir.glob('*.jpg')))
expected_files = len(df_train)

print(f"\n📊 全体統計:")
print(f"   期待ファイル数: {expected_files:,}")
print(f"   実際ファイル数: {total_files:,}")
print(f"   一致率: {100*total_files/expected_files:.2f}%")

if total_files == expected_files:
    print(f"\n✅ 全サンプルのクロップに成功しました！")
else:
    print(f"\n⚠️ {expected_files - total_files}ファイルが不足しています")

## 6. サンプル画像確認

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

# ランダムに10サンプル選択
sample_indices = np.random.choice(len(df_train), size=min(10, len(df_train)), replace=False)

fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()

for i, idx in enumerate(sample_indices):
    crop_path = output_dir / f"{idx}.jpg"
    
    if crop_path.exists():
        img = cv2.imread(str(crop_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        row = df_train.iloc[idx]
        label = row['label_id']
        angle = row['angle']
        
        axes[i].imshow(img)
        axes[i].set_title(f"ID:{idx} | Player:{label} | {angle}", fontsize=9)
        axes[i].axis('off')
    else:
        axes[i].text(0.5, 0.5, 'Not Found', ha='center', va='center')
        axes[i].axis('off')

plt.suptitle('クロップ済み画像サンプル（ランダム10枚）', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("✅ サンプル画像表示完了")